# PROJETO INTERDISCIPLINAR PARA SISTEMAS DE INFORMAÇÃO IV
## PROJETO FINAL

*Aluna: Marília Gouveia Rodrigues Costa*

__________________________

#### INFORMAÇÕES BÁSICAS SOBRE O PROJETO

##### APLICAÇÕES ESCOLHIDAS

- Aplicação de **reconhecimento facial ou de objetos** utilizando o **Amazon Rekognition**;
- Criar um serviço **Text-to-Speech (TTS)** utilizando **Amazon Polly**;

##### OBJETIVOS DO PROJETO

- ###### OBJETIVO GERAL

Desenvolvimento de um sistema para **renhecimento de fotos de pessoas famosas**, que reconheça a pessoa e reproduza seu nome em um alto-falante.

- ###### OBJETIVO ESPECÍFICOS

Desenvolvimento de um sistema capaz de reconhecer um ou mais famosos em fotos e reproduzir seu nome em um alto-falante.

__________________________

#### CÓDIGO

##### IMPORTAÇÃO DE PACOTES DO PYTHON

Importação de pacotes do Python que serão necessários para o desenvolvimento do projeto.

- *boto3*: Kit de desenvolvimento de software (SDK) da Amazon Web Services (AWS) para Python, que tem como objetivo facilitar a integração do script Python aos serviços da AWS;
- *matplotlib*: Fornecimento de funções de plotagem;
- *PIL*: Biblioteca de imagens do Python, que possui um conjunto de ferramentas destinadas ao desenho de imagens;

In [69]:
import boto3

from matplotlib import pyplot as plt

from PIL import Image, ImageDraw, ImageColor, ImageOps

##### CRIAÇÃO DE CLIENTES DE SERVIÇOS

Cria clientes de serviços do Amazon Rekognition e do Amazon Polly.

- *Criação,  por nome , de clientes de serviços de baixo nível usando a sessão padrão.*

In [70]:
client_amzrekognition = boto3.client('rekognition') # Amazon Rekognition
client_amzpolly = boto3.client('polly') # Amazon Polly

##### DEFINIÇÃO DA FUNÇÃO RESPONSÁVEL PELO RECONHECIMENTO DAS(S) CELEBRIDADE(S)

Define a função resposável por reconhecer o(s) artista(s), presente(s) na foto passada como parâmetro, e indicar quantos foram encontrados, o nome e o id dentro do serviço de reconhecimento de celebridades do Amazon Rekognition de cada, caso haja mais de um, e a(s) posições da(s) caixa(s) delimitadora(s) na imagem.

In [71]:
def recognize_celebrities(celebrity_photo):
    
    #Abre o arquivo que é passado como parâmetro como uma imagem.
    with open(celebrity_photo, "rb") as image:
        
        # Retona uma matriz de celebridades reconhecidas na imagem passada.
        celebrities = client_amzrekognition.recognize_celebrities (Image = { 
            "Bytes": image.read()
        })
    
    # Retorna um array contendo todas as informações analisadas na foto e existentes no serviço, sobre cada celebridade reconhecida.
    return celebrities['CelebrityFaces']
    

##### RECEBIMENTO DA FOTO DO USUÁRIO

Solicita ao usuário do nome do arquivo da foto que deseja analisar.

In [73]:
photo = "celebrities_photo2.jpg"

##### RECONHECIMENTO DAS(S) CELEBRIDADE(S)

Chama a função recognize_celebrities(celebrity_photo), definida anteriormente, resposável pela indentificação das faces na imagem e do recomhecimento das celebridades.

In [74]:
recognition = recognize_celebrities(photo)

##### APRESENTAÇÃO DAS INFORMAÇÕES SOBRE O RECONHECIMENTO

Define uma lista vazia.

Essa lista será responsável por armazenar o(s)  nome(s) da(s) celebridade(s) reconhecidas.

In [75]:
celebrities_names = []

Imprime na tela informações sobre:

- O RECONHECIMENTO DAS CELEBRIDADES: Quantidade de celebridades reconhecidas e a(s) posições da(s) caixa(s) delimitadora(s), que foram definidas, para cada face reconhecida, durante a análise da imagem;
- AS CELEBRIDADES: Nome e o id dentro do serviço de reconhecimento de celebridades do Amazon Rekognition de cada, caso haja mais de um;

In [78]:
# Imprime o nome do arquivo que está sendo analisada.   
print("DETECÇÃO DE FACES PRESENTES NA IMAGEM " + "'" + photo + "'" + "\n")
    
# Imprime a quantidade de faces que foram identificadas no reconhecimento.
print("CELEBRIDADES DETECTADAS: " + str(len(recognition)) + "\n")
        
#Imprime, para cada celebridade reconhecida, seu nome, id e a caixa delimitadora.
for celebrity in recognition:
    
    celebrities_names.append(celebrity['Name'])
    
    print ('Nome: ' + celebrity['Name'])
    print ('Id: ' + celebrity['Id'])
    print ('Posição na Imagem:')
    print ('   Esquerda: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Height']))
    print ('   Superior: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Top']) + "\n")
       

DETECÇÃO DE FACES PRESENTES NA IMAGEM 'celebrities_photo2.jpg'

CELEBRIDADES DETECTADAS: 3

Nome: Tom Hiddleston
Id: 39Ee4N
Posição na Imagem:
   Esquerda: 0.13
   Superior: 0.43

Nome: Benedict Cumberbatch
Id: cv9Gc9
Posição na Imagem:
   Esquerda: 0.12
   Superior: 0.47

Nome: Tom Holland
Id: ua51E
Posição na Imagem:
   Esquerda: 0.10
   Superior: 0.51



##### APRESENTAÇÃO DOS NOMES DAS CELEBRIDADES DETECTADAS

Imprime em sequência o nome de todas as celebridades que foram reconhecidas durante a análise da foto.

In [77]:
for names in celebrities_names:
    print(names)

Tom Hiddleston
Benedict Cumberbatch
Tom Holland
